In [4]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
#from visual import *
import numpy as np
import timeit
from math import *

In [10]:
def initialize(speed = 1, N = 100, width = 1000, height = 1000):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    agents = [Point(width*(0.25+0.5*random()), height*(0.25+0.5*random())) for i in range(N)]
    speeds = [np.array([0.0, 0.0]) for i in range(N)]
    for i in range(N):
        theta = 2 * np.pi * random()
        speeds[i][0] = speed * np.cos(theta)
        speeds[i][1] = speed * np.sin(theta)

    return agents, speeds, draw_agents(agents, width, height)


def draw_agents(agents, width, height):
    win = GraphWin("Swarm", width, height) # size of box
    for agent in agents:
        agent.draw(win)
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
            weightedSpeed = a * speeds[nearest_neighbours[i]]
            speeds[i] = speeds[i] + weightedSpeed
            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------


def get_distances(agent, agents):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    
    dists = [(a.getX() - agent.getX())**2 + (a.getY() - agent.getY())**2 for a in agents]
    for i in range(len(dists)):
        if dists[i] == 0:
            dists[i] = 0.1
    return dists


def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def treat_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)
        
        
def periodic_boundary(x_bound, y_bound, agents, speeds, N):  #Changed from rigid boundaries do periodic boundary condition
    [dx, dy] = [0, 0]
    for i in range(N):
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            dx = -x_bound         
        
        elif x < 0:
            dx = x_bound
        
        if y > y_bound:
            dy = -y_bound
        
        elif y < 0:
            dy = y_bound

        agents[i].move(dx, dy)

def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        agents[i].move(dxvec[i], dyvec[i])
        

def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


#def get_v_avg(speeds, N):
#    return
#   
#def get_potencials(distances, N):
#    potentials = np.zeros(N, N)
#    for i in range(N):
#        for j in range(i+1,N):
#            potentials[i][j] = potentials[j][i] = morse_pot(distances[i][j])
#    return potentials  
#    
#def morse_pot(r, ca = 0.5, cr = 1, la = 3, lr = 0.5):
#    return cr*np.exp(-r/lr) - ca * np.exp(-r)
#
#def weird_paper(agents, N, dt):
#    distances = [get_distances(agent, agents) for agent in agents]
#    potencials = get_potencials(distances, N)
#    for i in range(N):
#            newSpeed = speed[i] + dt * potentials
#            speeds[i] = speeds[i] + weightedSpeed
#            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------
#
#    return

In [22]:
#COUZIN MODEL IMPLEMENTED WITH REPULSE, ATRACT AND ORIENT ZONES SEPARATED (1ST PAPER)
def couzin(agents, speeds, N, s, noise, rr, ro, ra):
    # watch only particles inrepuls
    for i in range(N): ### FIX - Eliminate, in some way the i-i interaction
        curr_agent = agents[i]
        distances = get_distances(curr_agent, agents)
        r_dir = np.array([0.0, 0.0])
        o_dir = np.array([0.0, 0.0])
        a_dir = np.array([0.0, 0.0])
        repulsion_flag = False    
        
        for j in range(N):
            if i == j:
                continue

            if distances[j] < rr:
                temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                temp_vec = normalized(temp_vec)
                r_dir = r_dir + temp_vec
                repulsion_flag = True
            
            elif not repulsion_flag:

                if distances[j] < ro:
                    o_dir = o_dir + speeds[j]
                
                elif distances[j] < ra:
                    temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                    temp_vec = normalized(temp_vec)
                    a_dir = a_dir + temp_vec
            
        #Out of for (j), we treat now the resulting direction vector
        if repulsion_flag:
            tot_dir =  normalized(-r_dir)
        else:
            o_dir = normalized(o_dir)
            a_dir = normalized(a_dir)
            tot_dir = o_dir + a_dir
            tot_dir = normalized(tot_dir)
            
        tot_dir = normalized(normalized(tot_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        
        #avoid pts stoping when not interacting
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir

#COUZIN MODEL (2) IMPLEMENTED WITH ATRACT AND ORIENT ZONES TOGETHER (2ND PAPER)
def couzin2(agents, speeds, N, s, noise, rr, roa, orient, atract):
    # watch only particles inrepuls
    for i in range(N): ### FIX - Eliminate, in some way the i-i interaction
        curr_agent = agents[i]
        distances = get_distances(curr_agent, agents)
        r_dir = np.array([0.0, 0.0])
        o_dir = np.array([0.0, 0.0])
        a_dir = np.array([0.0, 0.0])
        repulsion_flag = False    
        
        for j in range(N):
            if i == j:
                continue
            if distances[j] < rr:
                temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                temp_vec = normalized(temp_vec)
                r_dir = r_dir + temp_vec
                repulsion_flag = True
            elif not repulsion_flag:
                if distances[j] < roa:
                    o_dir = o_dir + speeds[j]
                    temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                    temp_vec = normalized(temp_vec)
                    a_dir = a_dir + temp_vec
        #Out of for (j), we treat now the resulting direction vector
        if repulsion_flag:
            tot_dir =  normalized(-r_dir)
        else:
            o_dir = normalized(o_dir)
            a_dir = normalized(a_dir)
            tot_dir = orient*o_dir + atract*a_dir
            tot_dir = normalized(tot_dir)
            
        tot_dir = normalized(tot_dir + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        
        #avoid pts stoping when not interacting
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir


def biaser(speeds, N, s, i, prop, bias, dev_bias, rot_bias, weight):
    #bias = np.array([0.0,1.0])
    #Ns has to be integer 
    Ns = int(N * prop)
    gbias = np.random.normal(bias, dev_bias, )
    for i in range(Ns):
        tot_dir = normalized(normalized(speeds[i]) + weight * gbias)
        
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir
    #bias = np.dot(tot_dir,np.array([[np.cos(rot_bias*i), 0],[0, np.sin(rot_bias*i)]]))
    return bias

def quality (agents, speeds, N, bias, window, old_cm):
#ACCURACY (DIRECTIONS DISTRIBUTION)
    dev = 0.0
    for i in range(N):
        dev += np.arccos(np.dot(bias,normalized(speeds[i]))) # stupid way of doing an angle
    dev_avg = dev/(N*2*(np.pi))
    #print (dev_avg)
    
#ELONGATION: SHAPE OF SWARM
    #can be smarter if we make agents become poses before, more globally in  the code
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    #Center of Mass
    cm = np.mean(poses,axis=0)
    #Standard Deviation
    std = np.std(poses,axis=0)
    #print (cm, std)

    #Elongation >1 means 
    elong = std[1] / std[0]
    #Drawing it
    #old_cm = Point(cm[0],cm[1])
    #old_cm.draw(window)
    #old_cm.setFill("red")

#GROUP DIRECTION
    #Vector
    group_dir = np.array([cm[0] - old_cm[0], cm[1] - old_cm[1]])                
    print (group_dir)
    # Norm
    group_dir = np.linalg.norm(group_dir)
    print (group_dir)
    return cm
    
def vicsek(agents, speeds, N, s, noise, r): # s=speed, noise= letter csi temperature factor, r=radius of interaction
    # consider only particles within 'r' from pt_i, align pt_i with v_avg
    for i in range(N):
        distances = get_distances(agents[i], agents)
        tot_dir = np.array([0.0, 0.0])
        
        for j in range(N):
            if distances[j] < r:
                tot_dir = tot_dir + speeds[j]

        tot_dir = s * normalized(normalized(tot_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = tot_dir

def gueron():
    return

In [29]:
def simulate(N_steps, a, dt, N, width, height, s, noise, prop, weight, bias, dev_bias, rot_bias):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth of window
    s - module of speed throughout agents

    """
    #Watch out!! format of old_cm was giving problems
    #old_cm=Point(0,0)
    #old_cm.setFill("red")
    old_cm=np.array([0.0,0.0])
    
    
    agents, speeds, window = initialize(s, N, width, height)
    for i in range(N_steps):
        next_step(agents, speeds, dt, N)
        import timeit

        start = timeit.default_timer()
        
        ## MODEL CHOSEN (now with couzin 2 with orient and atract zones together)
        #couple_speeds(agents, speeds, a, s, N)
        #vicsek(agents, speeds, N, s, 0.1, r = 500)
        #couzin(agents, speeds, N, s, noise,200,2500,3000)
        couzin2(agents, speeds, N, s, noise,300,3100, 1.5,1) #(...)last 3 parameters: roa, orient, atract
        #EXTRA ON COUZIN MODEL
        bias = biaser(speeds, N, s, i, prop, bias, dev_bias, rot_bias, weight)
        #INFORMATION TRANSFER: SHAPE & DIRECTION & ALIGNMENT QUALITY 
        #point_cm.undraw()
        old_cm = quality(agents, speeds, N, bias, window, old_cm)
        #print (old_cm)
        
        ## BOUNDARY CONDITIONS
        #treat_boundary(width, height, agents, speeds, N)
        #periodic_boundary(width, height, agents, speeds, N)

        #time.sleep(0.02)
        stop = timeit.default_timer()
        #print stop - start
    window.close()
    

#simulate(N_steps, a, dt, N, width, height, s, noise, prop, weight, bias, dev_bias, rot_bias)
simulate(3000, 0.1, 2, 50, 900, 700, 2, 0.01, 0.4,0.2, np.array([0.0,-1.0]), 0.1, 1)

[ 444.10735082  340.50742422]
559.621876802
[-0.60839846  0.92407379]
1.10637292264
[-0.96367809  0.21662996]
0.987726682441
[-1.05394796  0.57348703]
1.19987236273
[-0.24500423  0.63999266]
0.685286569254
[-0.38479014  0.83778316]
0.921924112409
[ 0.02229269  0.87977583]
0.880058218056
[-0.36716984  0.48475123]
0.608109734369
[-1.02699128  0.28034475]
1.06456764226
[-1.33448752  0.09573151]
1.33791684179
[-1.00685687 -0.48950366]
1.11954213499
[-1.235324   -0.36340889]
1.28766898304
[-1.04391302 -0.34024483]
1.09796217674
[-0.9324952  -0.24869078]
0.965087773404
[-0.35509253 -0.275776  ]
0.449603277157
[-0.40695952 -0.549867  ]
0.684083161559
[-0.95778995 -0.86361214]
1.28964627929
[-0.5708941  -0.36919661]
0.679872202101
[-0.2104559  -0.31951958]
0.382602203733
[-0.16235698 -0.75373422]
0.771022088315
[-0.08073455 -0.62303138]
0.628240531537
[ 0.08086269 -0.11007809]
0.136586828303
[-0.0836654 -0.5394533]
0.545902699496
[ 0.19034533 -0.38966863]
0.433673825115
[ 0.4471963   0.3881995

[ 0.64270937  0.5872895 ]
0.870622928948
[-0.34066639  0.72507373]
0.801115162752
[-0.02331406  0.3496598 ]
0.350436190173
[ 0.04260922  0.61376121]
0.615238466215
[-0.16266969  0.20687574]
0.263171048591
[-0.20366552  0.38221105]
0.433087672539
[-0.23706561  0.7344969 ]
0.771806837655
[-0.4970708   0.30474076]
0.583049150952
[-0.24114668  0.04946703]
0.246168049315
[-1.47192349  0.44919771]
1.53894032948
[-1.04017723  0.12062213]
1.04714773204
[-1.21741105  0.81895473]
1.46723430569
[ 0.0085448   0.37810118]
0.378197719012
[ 0.36616015  0.49130289]
0.61274120659
[-0.11933046  1.02242687]
1.02936702341
[ 0.800531    0.51565825]
0.95223595281
[ 1.18919485  0.55245644]
1.31125607803
[ 1.25421479  0.76297874]
1.46805698273
[ 1.2887347   0.38673173]
1.34551052323
[ 0.79529748 -0.16762961]
0.812771660324
[ 0.72572894 -0.18102402]
0.747965367024
[ 0.88260212  0.20644213]
0.906424216341
[ 0.30407746 -0.32243384]
0.443200498428
[ 0.46190669 -0.42240372]
0.62592547149
[ 0.17224265 -0.83517227]


[ 0.95968755  0.71563404]
1.19713502779
[ 1.16942261  1.19429876]
1.67149596505
[ 0.81089035  1.26807405]
1.50517605281
[ 0.31250516  1.02877176]
1.07518873325
[-0.83284449  0.34063912]
0.899813842429
[-1.00095633 -0.38122872]
1.07109705975
[-1.19638344  0.14839722]
1.20555176629
[-1.65704886 -0.04168147]
1.65757300738
[-1.87170495  0.31272469]
1.89765017037
[-1.36879067 -0.13165714]
1.37510781182
[-0.67041943  0.57004917]
0.880010375293
[-1.15508116  0.52433954]
1.26852057134
[-0.57081043 -0.48854525]
0.751332823546
[-0.70044984 -0.51701128]
0.870592118157
[-0.38068621  0.10447894]
0.394763014023
[-0.24660999 -0.32204703]
0.405623936468
[-0.22143532 -0.63000682]
0.667789034905
[-1.00742984 -0.91932617]
1.36384584477
[-0.21091742 -0.30587107]
0.371541745975
[-0.19720084 -1.09950892]
1.1170532805
[ 0.07030983 -0.75962825]
0.762875182384
[ 0.41714491 -0.90452936]
0.996083949025
[ 0.65094474 -1.44276618]
1.5828149934
[ 0.26748281 -1.48946568]
1.5132927857
[-0.08331151 -1.67335859]
1.67543

[-0.89403407 -0.54952423]
1.04941592818
[-1.42336938 -0.61065677]
1.54883249206
[-1.59066351 -0.73337062]
1.75158296156
[-1.13091976 -0.98388774]
1.49900453809
[-0.75141186 -0.92865079]
1.19457610136
[-1.35151591 -0.36856096]
1.40086845793
[-1.09160895 -0.61998275]
1.25538389045
[-0.35869057 -1.01291201]
1.07454626312
[ 0.10696229 -1.50213123]
1.50593465256
[ 0.04010731 -1.14816576]
1.14886604978
[-0.11083603 -1.84016995]
1.84350483715
[-0.3920289  -1.59846518]
1.64583644211
[-0.60427869 -1.98718821]
2.07703387399
[-0.61441075 -1.75018758]
1.85490084407
[-0.74891097 -1.39414073]
1.58255995393
[-0.41608318 -1.30405072]
1.36882193582
[ 0.15788221 -1.07665976]
1.08817417568
[ 0.59602638 -0.2846112 ]
0.660492978091
[ 0.93464516 -0.50791678]
1.0637391742
[ 0.52197754 -1.01132652]
1.13808694026
[ 0.76209342 -0.77583911]
1.08752596045
[ 0.56528781 -0.22843134]
0.609697612962
[ 0.48091728 -0.29882971]
0.566198399569
[-0.26619238 -0.73951932]
0.785968962951
[ 0.99880107 -1.12422604]
1.503824382

[-0.64202628 -0.81833184]
1.04012726966
[-0.26005468 -0.68734009]
0.734891031577
[-0.64275936 -0.01421796]
0.642916594814
[-0.74400839 -0.36865004]
0.830332068773
[-0.54658307 -0.8452846 ]
1.00660771988
[-0.70472927 -0.96849151]
1.1977558785
[-0.39949275 -0.69416688]
0.800913302865
[ 0.51412941 -0.62342871]
0.80807945427
[ 0.65642133 -0.32259218]
0.731405953715
[ 0.19110288 -0.18043848]
0.262827616616
[ 0.54995398  0.00098849]
0.549954868852
[ 0.73984048 -0.40826049]
0.845009207322
[ 0.51650953 -0.57719995]
0.774559150811
[ 0.36255389 -0.55493547]
0.662871555782
[ 0.97883254 -0.40152092]
1.05798496562
[ 0.84515247 -0.61023297]
1.04243319843
[ 0.58404214 -1.49302035]
1.60318900644
[ 0.83332539 -1.58910705]
1.79435013565
[ 0.8489541  -1.20852575]
1.47690810769
[ 0.45710749 -0.75679216]
0.884127610023
[-0.02959519 -0.44659167]
0.447571219034
[ 0.83861045 -0.67552286]
1.07684660539
[ 0.41978763 -0.51188809]
0.662005339793
[-0.70702494 -0.76157534]
1.03917335598
[-1.30967055 -0.89213555]
1.

...

[-1.50132179 -0.34503958]
1.54046078084
[-0.86678518 -0.95396233]
1.2889378079
[-0.85497312 -1.20230077]
1.47529866768
[-0.539015   -1.93070809]
2.00453757445
[-0.16520908 -1.55478932]
1.5635420931
[-0.18769378 -1.37905626]
1.3917704963
[ 0.43407871 -2.0646935 ]
2.10983022162
[ 0.47213883 -1.49911364]
1.57170505785
[ 0.12891627 -1.96576312]
1.9699858024
[ 0.50235952 -2.30866655]
2.36269048719
[-0.01467217 -1.76745045]
1.76751134723
[-0.85845792 -1.68001769]
1.88663972159
[-1.0645765  -0.91287027]
1.40237486544
[-1.57935171 -0.31377993]
1.6102203759
[-1.69059585 -0.36223511]
1.72896749211
[-1.31052501 -0.08032141]
1.31298413269
[-1.46147489  0.61173437]
1.58433827874
[-1.31519708  0.45146401]
1.39052620013
[-1.33503668  0.02765188]
1.33532302281
[-1.23607398  0.5335257 ]
1.34630180825
[-1.15846472  0.59642121]
1.3029807271
[-1.28776769 -0.2031022 ]
1.30368559659
[-1.0591316  -0.34112572]
1.11271132942
[-1.62963517 -0.13083436]
1.63487871367
[-1.19006467 -0.09061498]
1.1935095312
[-1.457

[-0.74872535 -0.21503681]
0.778993248637
[-1.17796588 -0.17194049]
1.19044829853
[-0.70342405 -0.28816867]
0.760162201282
[-0.70071917  0.04590092]
0.702220938303
[-0.91889027 -0.40226963]
1.00308532954
[ 0.0736367  -1.05419822]
1.05676688808
[-0.07655267 -0.66651006]
0.670891926019
[-0.51915484 -0.32769597]
0.613927028939
[-0.38505916 -0.58206355]
0.697902952665
[ 0.22094384 -1.04315917]
1.06630072361
[ 0.0272885  -0.19034575]
0.192291883501
[ 0.82336096  0.28800815]
0.872279753823
[ 1.04536498 -0.62471567]
1.21780852857
[ 0.29409586 -0.66578038]
0.727843315101
[ 0.31301631 -0.26405467]
0.409516883811
[ 0.59990175 -0.79383178]
0.995013072013
[ 0.27870727 -0.51510267]
0.5856692756
[ 0.42467607 -1.2150845 ]
1.28715970213
[ 0.07003038 -0.31404391]
0.321757410364
[ 0.48674728 -0.28325041]
0.563164013574
[ 0.55585818 -0.37931224]
0.67294582856
[ 0.59503034 -0.33599972]
0.683342459707
[ 0.83641963 -0.75751488]
1.12846204606
[ 0.38775132 -0.99549105]
1.06834148205
[-0.02709774 -0.61535094]
0

[ 0.37109486  0.45648789]
0.588296347266
[ 0.03243948  0.97429164]
0.974831537125
[ 0.2852174  -0.14757274]
0.321133429799
[ 0.18901403 -0.09294171]
0.210628736268
[-0.05925696 -0.96441022]
0.966228985861
[ 0.09620589 -0.99571683]
1.0003537227
[ 0.03627282 -0.67553606]
0.676509192717
[-0.31102236  0.11936887]
0.333142359608
[-0.33933974 -0.3535021 ]
0.490015503711
[-0.76098199 -0.80496466]
1.10772816819
[-0.99667479 -0.20798134]
1.01814383577
[-0.81254546 -0.67139058]
1.05403768036
[-0.88590619 -0.99209556]
1.33006893637
[-0.61501524 -0.60103236]
0.859932344941
[-1.52060632 -0.53574132]
1.61222279846
[-1.17009103 -0.55080193]
1.29325008859
[-0.86921378 -0.50987359]
1.00772202409
[-0.66864499 -0.47974164]
0.822944809222
[-0.71409532 -0.27520791]
0.765291790884
[-0.06103582 -1.00215587]
1.00401283015
[ 0.28059879 -0.45032259]
0.53059034679
[ 0.16834278 -0.26811552]
0.316583678463
[-0.12157442 -0.43378576]
0.450500193579
[ 0.0387383   0.58774579]
0.589021026235
[ 0.00830174 -0.147232  ]
0

[-0.37099865 -0.51998284]
0.638766114801
[-0.74113394 -0.46530814]
0.875094956831
[-0.73635797 -0.20973685]
0.765645222151
[-0.70799245 -0.20436426]
0.736897590482
[-0.51878906 -0.21893703]
0.563094589778
[-0.32126485 -0.77654283]
0.840374841133
[-0.41002445 -0.87029823]
0.962049404003
[-0.09371478 -1.10381909]
1.10779016191
[-0.41727017 -1.20242493]
1.27276867975
[ 0.22050647 -0.88578888]
0.912822568958
[ 0.58415584 -1.49391875]
1.60406710354
[ 0.62115242 -1.98148885]
2.07656649247
[ 1.11189865 -1.35220492]
1.75065037759
[ 0.61387796 -1.21161392]
1.35825418793
[-0.07980804 -0.86321934]
0.86690077435
[-0.9055771  -0.43972379]
1.00669105656
[-0.88439544 -0.20665821]
0.908219638373
[-0.42173344 -0.305099  ]
0.52052328471
[-0.43526037 -0.48308611]
0.65024901204
[-0.76426623 -0.27640795]
0.812714109583
[-1.01738643 -1.17534762]
1.55451508924
[-1.03668256 -0.46828951]
1.13754375355
[-0.96419814 -0.80088705]
1.25343452751
[-0.4777295  -0.82733084]
0.955354276864
[-0.37960547 -0.33987382]
0.5

[ 0.56330027  0.39806067]
0.689753210894
[-0.7442822   0.75856225]
1.06271947562
[-0.58568097  1.11034935]
1.25534771026
[-0.4395398   1.04486718]
1.13355311288
[-1.14261704  0.91504305]
1.46385706001
[-0.63288555  0.92860063]
1.12376298641
[ 0.13255083 -0.23056861]
0.26595414157
[ 0.93362471 -1.25661971]
1.56548657177
[ 1.13668188 -1.38893543]
1.7947666555
[ 1.53332764 -1.43116262]
2.0974556288
[ 1.1832139  -1.45268158]
1.87357382948
[ 0.75993171 -0.90925406]
1.18500597096
[ 0.33646163 -1.34149616]
1.38304677341
[ 0.67343867 -1.41544336]
1.56748191271
[ 0.48804093 -1.02154489]
1.13213864412
[ 0.3785315  -0.95621627]
1.02841414092
[ 0.8496503  -1.17285688]
1.44827444837
[ 1.15673589 -1.69259863]
2.05010439847
[ 0.6501185  -1.05211028]
1.23676598356
[ 1.10433618 -0.63503048]
1.27390035357
[ 0.19954519 -0.91193077]
0.933507375776
[ 0.21938893 -0.84477125]
0.872794342136
[ 0.67770227 -0.6659351 ]
0.95013152796
[ 0.86482633 -0.28221619]
0.909709054933
[ 0.32884052 -0.44949407]
0.5569389585